### 1.ライブラリのインポート

In [ ]:
#一度pip install したらコメントアウトしてください！！
# !pip install japanize-matplotlib 
# !pip install tabulate
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import sys
import os


### 2.データの読み込みと前処理

In [ ]:
# データの読み込みと前処理 (エンコーディング修正済み)
# ファイル格納場所に応じてパスを変える
# パターン1: work_datasetフォルダ内にある時
path_pattern_1 = 'work_dataset/stats_bat.csv'

# パターン2: 実行ファイルと同じ階層の時
path_pattern_2 = 'stats_bat.csv'

# どちらのパスを使うか決定するための変数
use_file_path = None

# ファイルがあるか確認 
if os.path.exists(path_pattern_1):
    use_file_path = path_pattern_1
    print(f"パターン1のパスを採用: {use_file_path}")

elif os.path.exists(path_pattern_2):
    use_file_path = path_pattern_2
    print(f"パターン2のパスを採用: {use_file_path}")

else:
    # どちらも見つからなかったとき
    print("エラー: 'stats_bat.csv' が見つかりません。")
    print(f"以下の場所を確認してください:")
    print(f" - {path_pattern_1}")
    print(f" - {path_pattern_2}")
    # プログラムをここで終了するか、処理をスキップ
    use_file_path = None


# 読み込み処理 (のちの処理のために変数名を変更)
file_name = use_file_path
try:
    # 最初にShift-JISで読み込みを試みる
    df = pd.read_csv(file_name, encoding='Shift-JIS')
    print(":white_check_mark: Shift-JISでファイルを読み込みました。")
except UnicodeDecodeError:
    try:
        # Shift-JISで失敗した場合、cp932 (WindowsのShift-JIS)で試す
        df = pd.read_csv(file_name, encoding='cp932')
        print(":white_check_mark: cp932でファイルを読み込みました。")
    except Exception as e:
        print(f":x: ファイルの読み込みに失敗しました。エンコーディングエラー: {e}")
        sys.exit() # 読み込みエラーでプログラムを終了

# 必要なカラムを選択
columns_to_keep = ['player_id', 'player_name', 'pa', 'ab', 'h', 'tb', 'bb', 'hbp', 'sf', 'so', 'hr']
df_bat = df[columns_to_keep].copy()

# 規定打席数（今回は100打席以上）未満の選手を除外
MIN_PA = 100
df_bat = df_bat[df_bat['pa'] >= MIN_PA].reset_index(drop=True)

print(f":white_check_mark: データ読み込み完了。規定打席数 ({MIN_PA} PA) 以上の選手数: {len(df_bat)}名")
print("-" * 30)

#dfの確認
df_bat.info()#データフレームの情報(カラム名)の確認

### 3.ops計算

In [ ]:
# 出塁率 (OBP) = (H + BB + HBP) / (AB + BB + HBP + SF)
df_bat['OBP_denominator'] = df_bat['ab'] + df_bat['bb'] + df_bat['hbp'] + df_bat['sf']
df_bat['OBP_numerator'] = df_bat['h'] + df_bat['bb'] + df_bat['hbp']
df_bat['OBP'] = np.where(df_bat['OBP_denominator'] > 0, df_bat['OBP_numerator'] / df_bat['OBP_denominator'], np.nan)

# 長打率 (SLG) = TB / AB
df_bat['SLG'] = np.where(df_bat['ab'] > 0, df_bat['tb'] / df_bat['ab'], np.nan)

# OPS = OBP + SLG
df_bat['OPS'] = df_bat['OBP'] + df_bat['SLG']


#df確認
# df_bat.head() #先頭の５行を表示して確認 
df_bat.info()#新しい情報が追加されているか確認

### 4.分析指標の計算

In [ ]:
# 三振率 K% = SO / PA
df_bat['K_Rate'] = df_bat['so'] / df_bat['pa']

# ホームラン率 HR/PA = HR / PA
df_bat['HR_Rate'] = df_bat['hr'] / df_bat['pa']

df_bat.dropna(subset=['OPS', 'K_Rate', 'HR_Rate'], inplace=True)

print(":white_check_mark: OPSおよび分析指標の計算完了。")
print("-" * 30)

#df確認
df_bat.head()

### 5.相関分析と可視化
- ※ここでは分析結果の概要出力とグラフのファイル保存のみ行います

#### 5-A. OPSと三振率の関係

In [ ]:
# 日本語表示のための設定 (環境によって適宜調整してください)
plt.rcParams['font.family'] = 'Hiragino Maru Gothic Pro'  
plt.rcParams['axes.unicode_minus'] = False # マイナス記号の表示

corr_k_ops = df_bat['K_Rate'].corr(df_bat['OPS'])
print(f"三振率 (K_Rate) と OPS の相関係数: {corr_k_ops:.4f}")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='K_Rate', y='OPS', data=df_bat, alpha=0.6)
plt.title(f'OPSと三振率の関係 (相関係数: {corr_k_ops:.4f})')
plt.xlabel('三振率 (SO / PA)')
plt.ylabel('OPS')
plt.savefig('ops_vs_k_rate.png')
plt.close()

#### 5-B.OPSとホームラン率の関係

In [ ]:
corr_hr_ops = df_bat['HR_Rate'].corr(df_bat['OPS'])
print(f"ホームラン率 (HR_Rate) と OPS の相関係数: {corr_hr_ops:.4f}")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='HR_Rate', y='OPS', data=df_bat, alpha=0.6)
plt.title(f'OPSとホームラン率の関係 (相関係数: {corr_hr_ops:.4f})')
plt.xlabel('ホームラン率 (HR / PA)')
plt.ylabel('OPS')
plt.savefig('ops_vs_hr_rate.png')
plt.close()

#### 5-C.高三振・高OPSバッターの確認

In [ ]:
k_rate_q3 = df_bat['K_Rate'].quantile(0.75)
high_k_high_ops_players = df_bat[df_bat['K_Rate'] >= k_rate_q3].sort_values(by='OPS', ascending=False)
print(f"### 三振率が上位25% (K_Rate >= {k_rate_q3:.4f}) のバッター上位10名 (OPS順) ###")
print(high_k_high_ops_players[['player_name', 'pa', 'so', 'hr', 'K_Rate', 'HR_Rate', 'OBP', 'SLG', 'OPS']].head(10).to_markdown(index=False, floatfmt=".3f"))
print("-" * 30)


### 6.データベースへの挿入

In [ ]:
db_name = 'baseball_analysis.db'
table_name = 'batter_performance_ops'
conn = None

try:
    conn = sqlite3.connect(db_name)
    print(f":white_check_mark: データベース '{db_name}' に接続しました。")

    # 挿入するデータフレームを、必要なカラムだけに絞る
    df_result = df_bat[['player_id', 'player_name', 'pa', 'so', 'hr', 'K_Rate', 'HR_Rate', 'OBP', 'SLG', 'OPS']].copy()
    
    # DBへの書き込み (テーブルが既に存在する場合は置き換える)
    df_result.to_sql(
        name=table_name, 
        con=conn, 
        if_exists='replace', 
        index=False 
    )
    
    print(f":white_check_mark: 分析結果をテーブル '{table_name}' に挿入しました。")
    
    # 確認のために、DBから最初の5行を読み出して表示
    df_check = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", conn)
    print("\n--- 挿入されたデータの確認 (最初の5行) ---")
    print(df_check.to_markdown(index=False, floatfmt=".3f"))
    
except Exception as e:
    print(f"データベース処理中にエラーが発生しました: {e}")

finally:
    if conn:
        conn.close()
        print("\n:white_check_mark: データベース接続を閉じました。")

### 7.血液型と年齢の分析

In [ ]:
# ステップ1：players.csv を読み込み、打撃成績データと結合する
# players.csv の読み込み（年齢・血液型・年俸などの個人情報が含まれる）
players = pd.read_csv('players.csv', encoding='cp932')

# 誕生日データを日時型に変換
players['birthday_date'] = pd.to_datetime(players['birthday_date'], errors='coerce')

# df_bat（打撃成績データ）と players（個人情報）を player_id で結合
merged = pd.merge(df_bat, players, on='player_id', how='left')

print("結合後のデータ数:", merged.shape)
merged.head()

# ステップ2：各選手の“シーズン年齢”を計算する
# シーズン年齢を計算する関数
# 例：2023年の成績 → 2023 - 誕生年 = 年齢
# ステップ2：各選手の“シーズン年齢”を計算する

# どの season_year カラムを使うか自動で判定（season_year_x / season_year_y など）
season_cols = [c for c in merged.columns if 'season_year' in c]
if len(season_cols) > 0:
    season_col = season_cols[0]  # 先頭のものを使用
else:
    season_col = 'season_year'   # 念のため

print("シーズン年に使用するカラム:", season_col)

# シーズン年齢を計算する関数
def calc_age(row):
    if pd.isnull(row['birthday_date']):
        return None
    return row[season_col] - row['birthday_date'].year

# 年齢列を追加
merged['age'] = merged.apply(calc_age, axis=1)

# player_name と season_year のカラム名を自動判定してプレビュー
name_cols = [c for c in merged.columns if 'player_name' in c]
season_cols = [c for c in merged.columns if 'season_year' in c]

print("player_name 系カラム:", name_cols)
print("season_year 系カラム:", season_cols)

name_col = name_cols[0] if len(name_cols) > 0 else None
season_col_view = season_cols[0] if len(season_cols) > 0 else None

if name_col is not None and season_col_view is not None:
    display(merged[[name_col, season_col_view, 'age']].head())
else:
    print("player_name または season_year のカラムが見つからなかったため、プレビューをスキップしました。")

# ステップ3：年齢と OPS の関係（散布図 + 相関係数）
import seaborn as sns
import matplotlib.pyplot as plt

# 年齢と OPS のデータを抽出し、欠損を除外
age_ops = merged[['age', 'OPS']].dropna()

# 年齢と OPS の相関係数を計算
corr_age_ops = age_ops['age'].corr(age_ops['OPS'])
print(f"年齢とOPSの相関係数: {corr_age_ops:.4f}")

# 散布図の作成
plt.figure(figsize=(8,6))
sns.scatterplot(data=age_ops, x='age', y='OPS', alpha=0.6)
plt.title(f'年齢とOPSの関係 (相関係数: {corr_age_ops:.4f})')
plt.xlabel('年齢')
plt.ylabel('OPS')
plt.tight_layout()
plt.savefig('age_vs_ops.png', dpi=300, bbox_inches='tight')
plt.close()

# ステップ4：血液型と OPS の平均値を比較（棒グラフ）
# 血液型ごとの OPS 平均値を計算
blood_ops = merged.groupby('blood_type')['OPS'].mean().dropna()

# 棒グラフの描画
plt.figure(figsize=(6,4))
blood_ops.plot(kind='bar')
plt.title('血液型別のOPS平均')
plt.xlabel('血液型')
plt.ylabel('OPS')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('bloodtype_ops.png', dpi=300, bbox_inches='tight')
plt.close()

print(blood_ops)

# ステップ5：血液型と年俸の平均値を比較（棒グラフ）
# 血液型ごとの年俸平均を計算
blood_salary = merged.groupby('blood_type')['salary'].mean().dropna()

# 年俸の棒グラフを描画
plt.figure(figsize=(6,4))
blood_salary.plot(kind='bar')
plt.title('血液型別の年俸平均')
plt.xlabel('血液型')
plt.ylabel('年俸')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('bloodtype_salary.png', dpi=300, bbox_inches='tight')
plt.close()

print(blood_salary)

# ステップ6（オプション）：血液型 × OPS × 年俸 のまとめ表
# 血液型ごとの OPS と 年俸 の平均をまとめて表示
summary_blood = merged.groupby('blood_type')[['OPS', 'salary']].mean()
summary_blood